# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe8200169d0>
├── 'a' --> tensor([[ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449]])
└── 'x' --> <FastTreeValue 0x7fe820016850>
    └── 'c' --> tensor([[ 0.5392,  0.4167,  2.5625, -0.8599],
                        [ 0.9253, -1.0235, -0.2453,  1.8752],
                        [-1.3958, -1.4544, -0.8702,  0.4972]])

In [4]:
t.a

tensor([[ 0.6980, -0.6467, -0.0737],
        [-2.1048, -1.3671,  0.5449]])

In [5]:
%timeit t.a

73.9 ns ± 0.602 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe8200169d0>
├── 'a' --> tensor([[ 0.3370, -2.2051, -0.0134],
│                   [-0.3465, -0.1783, -0.5282]])
└── 'x' --> <FastTreeValue 0x7fe820016850>
    └── 'c' --> tensor([[ 0.5392,  0.4167,  2.5625, -0.8599],
                        [ 0.9253, -1.0235, -0.2453,  1.8752],
                        [-1.3958, -1.4544, -0.8702,  0.4972]])

In [7]:
%timeit t.a = new_value

95.9 ns ± 1.62 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6980, -0.6467, -0.0737],
               [-2.1048, -1.3671,  0.5449]]),
    x: Batch(
           c: tensor([[ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6980, -0.6467, -0.0737],
        [-2.1048, -1.3671,  0.5449]])

In [11]:
%timeit b.a

80.4 ns ± 1.42 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4610,  0.7039, -0.2571],
               [ 0.7490,  0.7873, -0.0675]]),
    x: Batch(
           c: tensor([[ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972]]),
       ),
)

In [13]:
%timeit b.a = new_value

627 ns ± 12.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.08 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.2 µs ± 163 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

284 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

269 µs ± 4.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe775cccfd0>
├── 'a' --> tensor([[[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]],
│           
│                   [[ 0.6980, -0.6467, -0.0737],
│                    [-2.1048, -1.3671,  0.5449]]])
└── 'x' --> <FastTreeValue 0x7fe775ccaf40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.3 µs ± 562 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe77c8eb6a0>
├── 'a' --> tensor([[ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449],
│                   [ 0.6980, -0.6467, -0.0737],
│                   [-2.1048, -1.3671,  0.5449]])
└── 'x' --> <FastTreeValue 0x7fe82008bfa0>
    └── 'c' --> tensor([[ 0.5392,  0.4167,  2.5625, -0.8599],
                        [ 0.9253, -1.0235, -0.2453,  1.8752],
                 

In [23]:
%timeit t_cat(trees)

42.2 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5392,  0.4167,  2.5625, -0.8599],
                       [ 0.9253, -1.0235, -0.2453,  1.8752],
                       [-1.3958, -1.4544, -0.8702,  0.4972]],
              
                      [[ 0.5392,  0.4167,  2.5625, -0.8599],
                       [ 0.9253, -1.0235, -0.2453,  1.8752],
                       [-1.3958, -1.4544, -0.8702,  0.4972]],
              
                      [[ 0.5392,  0.4167,  2.5625, -0.8599],
                       [ 0.9253, -1.0235, -0.2453,  1.8752],
                       [-1.3958, -1.4544, -0.8702,  0.4972]],
              
                      [[ 0.5392,  0.4167,  2.5625, -0.8599],
                       [ 0.9253, -1.0235, -0.2453,  1.8752],
                       [-1.3958, -1.4544, -0.8702,  0.4972]],
              
                      [[ 0.5392,  0.4167,  2.5625, -0.8599],
                       [ 0.9253, -1.0235, -0.2453,  1.8752],
                       [-1.3958, -1.4544, -0.8702,  0.4972]],

In [26]:
%timeit Batch.stack(batches)

102 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972],
                      [ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972],
                      [ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972],
                      [ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972],
                      [ 0.5392,  0.4167,  2.5625, -0.8599],
                      [ 0.9253, -1.0235, -0.2453,  1.8752],
                      [-1.3958, -1.4544, -0.8702,  0.4972],
                      [ 0.5392,  0.4167,  2.5625, -0.8599],
                   

In [28]:
%timeit Batch.cat(batches)

182 µs ± 3.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

605 µs ± 20.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
